Scenario 2: A cross-functional team with one data scientist working on an ML model
MLflow setup:

- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem
- The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

mlflow server --backend-store-uri sqlite:///backend.db



In [1]:

import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:

mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1717051995062, experiment_id='0', last_update_time=1717051995062, lifecycle_stage='active', name='Default', tags={}>]

In [4]:

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/30 09:01:58 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'mlflow-artifacts:/1/6db24df9432c4e1888d559db5f0213b4/artifacts'


In [5]:
mlflow.search_experiments()


[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1717052518692, experiment_id='1', last_update_time=1717052518692, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1717051995062, experiment_id='0', last_update_time=1717051995062, lifecycle_stage='active', name='Default', tags={}>]

Interacting with the model registry


In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [7]:
client.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1717052518692, experiment_id='1', last_update_time=1717052518692, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1717051995062, experiment_id='0', last_update_time=1717051995062, lifecycle_stage='active', name='Default', tags={}>]

Registering the model model 

In [12]:
import mlflow
experiment_id = '1'
runs = mlflow.search_runs(experiment_ids=[experiment_id])
run_id = runs.iloc[0].run_id
model_uri = f"runs:/{run_id}/models"
model_name = 'iris-classifier'

mlflow.register_model(
    model_uri=model_uri,
    name=model_name
)


'6db24df9432c4e1888d559db5f0213b4'